In [5]:
import autograd.numpy as np
from autograd import jacobian,hessian,grad
import autograd.numpy.random as npr
from matplotlib import cm
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d

## Set up the network

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def deep_neural_network(deep_params, x):
    # x is now a point and a 1D numpy array; make it a column vector
    num_coordinates = np.size(x,0)
    x = x.reshape(num_coordinates,-1)

    num_points = np.size(x,1)

    # N_hidden is the number of hidden layers
    N_hidden = np.size(deep_params) - 1 # -1 since params consist of parameters to all the hidden layers AND the output layer

    # Assume that the input layer does nothing to the input x
    x_input = x
    x_prev = x_input

    ## Hidden layers:

    for l in range(N_hidden):
        # From the list of parameters P; find the correct weigths and bias for this layer
        w_hidden = deep_params[l]

        # Add a row of ones to include bias
        x_prev = np.concatenate((np.ones((1,num_points)), x_prev ), axis = 0)

        z_hidden = np.matmul(w_hidden, x_prev)
        x_hidden = sigmoid(z_hidden)

        # Update x_prev such that next layer can use the output from this layer
        x_prev = x_hidden

    ## Output layer:

    # Get the weights and bias for this layer
    w_output = deep_params[-1]

    # Include bias:
    x_prev = np.concatenate((np.ones((1,num_points)), x_prev), axis = 0)

    z_output = np.matmul(w_output, x_prev)
    x_output = z_output

    return x_output[0][0]

## Define the trial solution and cost function
def u(x):
    return np.sin(np.pi*x)

def g_trial(point,P):
    x,t = point
    return (1-t)*u(x) + x*(1-x)*t*deep_neural_network(P,point)

# The right side of the ODE:
def f(point):
    return 0.

# The cost function:
def cost_function(P, x, t):
    cost_sum = 0

    g_t_jacobian_func = jacobian(g_trial)
    g_t_hessian_func = hessian(g_trial)

    for x_ in x:
        for t_ in t:
            point = np.array([x_,t_])

            g_t = g_trial(point,P)
            g_t_jacobian = g_t_jacobian_func(point,P)
            g_t_hessian = g_t_hessian_func(point,P)

            g_t_dt = g_t_jacobian[1]
            g_t_d2x = g_t_hessian[0][0]

            func = f(point)

            err_sqr = ( (g_t_dt - g_t_d2x) - func)**2
            cost_sum += err_sqr

    return cost_sum /( np.size(x)*np.size(t) )

## For comparison, define the analytical solution
def g_analytic(point):
    x,t = point
    return np.exp(-np.pi**2*t)*np.sin(np.pi*x)

## Set up a function for training the network to solve for the equation
def solve_pde_deep_neural_network(x,t, num_neurons, num_iter, lmb):
    ## Set up initial weigths and biases
    N_hidden = np.size(num_neurons)

    ## Set up initial weigths and biases

    # Initialize the list of parameters:
    P = [None]*(N_hidden + 1) # + 1 to include the output layer

    P[0] = npr.randn(num_neurons[0], 2 + 1 ) # 2 since we have two points, +1 to include bias
    for l in range(1,N_hidden):
        P[l] = npr.randn(num_neurons[l], num_neurons[l-1] + 1) # +1 to include bias

    # For the output layer
    P[-1] = npr.randn(1, num_neurons[-1] + 1 ) # +1 since bias is included

    print('Initial cost: ',cost_function(P, x, t))

    cost_function_grad = grad(cost_function,0)

    # Let the update be done num_iter times
    for i in range(num_iter):
        cost_grad =  cost_function_grad(P, x , t)

        for l in range(N_hidden+1):
            P[l] = P[l] - lmb * cost_grad[l]

    print('Final cost: ',cost_function(P, x, t))

    return P

if __name__ == '__main__':
    ### Use the neural network:
    npr.seed(15)

    ## Decide the vales of arguments to the function to solve
    Nx = 10; Nt = 10
    x = np.linspace(0, 1, Nx)
    t = np.linspace(0,1,Nt)

    ## Set up the parameters for the network
    num_hidden_neurons = [100, 25]
    num_iter = 250
    lmb = 0.01

    P = solve_pde_deep_neural_network(x,t, num_hidden_neurons, num_iter, lmb)

    ## Store the results
    g_dnn_ag = np.zeros((Nx, Nt))
    G_analytical = np.zeros((Nx, Nt))
    for i,x_ in enumerate(x):
        for j, t_ in enumerate(t):
            point = np.array([x_, t_])
            g_dnn_ag[i,j] = g_trial(point,P)

            G_analytical[i,j] = g_analytic(point)

    # Find the map difference between the analytical and the computed solution
    diff_ag = np.abs(g_dnn_ag - G_analytical)
    print('Max absolute difference between the analytical solution and the network: %g'%np.max(diff_ag))

    ## Plot the solutions in two dimensions, that being in position and time

    T,X = np.meshgrid(t,x)

    ## Take some slices of the 3D plots just to see the solutions at particular times
    indx1 = 0
    indx2 = int(Nt/2)
    indx3 = Nt-1

    t1 = t[indx1]
    t2 = t[indx2]
    t3 = t[indx3]

    # Slice the results from the DNN
    res1 = g_dnn_ag[:,indx1]
    res2 = g_dnn_ag[:,indx2]
    res3 = g_dnn_ag[:,indx3]

    # Slice the analytical results
    res_analytical1 = G_analytical[:,indx1]
    res_analytical2 = G_analytical[:,indx2]
    res_analytical3 = G_analytical[:,indx3]
   

Initial cost:  41.05505310046363
Final cost:  3.523938720339645
Max absolute difference between the analytical solution and the network: 0.362021


In [40]:
import pandas as pd

In [44]:
Neural_net_pd = pd.DataFrame(g_dnn_ag, columns = ["t=0","t = 0.1", "t = 0.2", "t=0.3", "t=0.4","t=0.5","t=0.6","t=0.7","t=0.8","t=0.9"])
Neural_net_pd

,t=0,t = 0.1,t = 0.2,t=0.3,t=0.4,t=0.5,t=0.6,t=0.7,t=0.8,t=0.9
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
1,3.420201e-01,2.379594e-01,1.453307e-01,6.944661e-02,1.456129e-02,-1.762659e-02,-2.886317e-02,-2.430186e-02,-1.107434e-02,0.004362
2,6.427876e-01,4.487634e-01,2.756915e-01,1.338537e-01,3.197769e-02,-2.651291e-02,-4.617949e-02,-3.886062e-02,-1.895689e-02,0.002300
3,8.660254e-01,6.063970e-01,3.746768e-01,1.849896e-01,4.967631e-02,-2.687861e-02,-5.258884e-02,-4.534076e-02,-2.466313e-02,-0.004204
4,9.848078e-01,6.908270e-01,4.287770e-01,2.148906e-01,6.330457e-02,-2.185087e-02,-5.136040e-02,-4.661662e-02,-2.909614e-02,-0.012795
5,9.848078e-01,6.909438e-01,4.299543e-01,2.180503e-01,6.886113e-02,-1.478122e-02,-4.501069e-02,-4.337207e-02,-3.031215e-02,-0.018730
6,8.660254e-01,6.063907e-01,3.772322e-01,1.925409e-01,6.338778e-02,-8.972568e-03,-3.585984e-02,-3.587517e-02,-2.625592e-02,-0.017899
7,6.427876e-01,4.479390e-01,2.774041e-01,1.411312e-01,4.644019e-02,-6.473853e-03,-2.604382e-02,-2.566326e-02,-1.787073e-02,-0.010852
8,3.420201e-01,2.363573e-01,1.447846e-01,7.222903e-02,2.209091e-02,-5.719394e-03,-1.548620e-02,-1.411162e-02,-8.322539e-03,-0.002751
9,1.224647e-16,1.088575e-16,9.525031e-17,8.164312e-17,6.803593e-17,5.442875e-17,4.082156e-17,2.721437e-17,1.360719e-17,0.000000


In [45]:
G_analytical_pd = pd.DataFrame(G_analytical, columns = ["t=0","t = 0.1", "t = 0.2", "t=0.3", "t=0.4","t=0.5","t=0.6","t=0.7","t=0.8","t=0.9"])
G_analytical_pd

,t=0,t = 0.1,t = 0.2,t=0.3,t=0.4,t=0.5,t=0.6,t=0.7,t=0.8,t=0.9
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,3.420201e-01,1.142338e-01,3.815376e-02,1.274325e-02,4.256209e-03,1.421562e-03,4.747976e-04,1.585811e-04,5.296563e-05,1.769037e-05
2,6.427876e-01,2.146893e-01,7.170561e-02,2.394947e-02,7.999056e-03,2.671662e-03,8.923276e-04,2.980349e-04,9.954282e-05,3.324702e-05
3,8.660254e-01,2.892500e-01,9.660870e-02,3.226703e-02,1.077710e-02,3.599521e-03,1.202230e-03,4.015414e-04,1.341137e-04,4.479359e-05
4,9.848078e-01,3.289230e-01,1.098594e-01,3.669272e-02,1.225526e-02,4.093224e-03,1.367125e-03,4.566160e-04,1.525085e-04,5.093739e-05
5,9.848078e-01,3.289230e-01,1.098594e-01,3.669272e-02,1.225526e-02,4.093224e-03,1.367125e-03,4.566160e-04,1.525085e-04,5.093739e-05
6,8.660254e-01,2.892500e-01,9.660870e-02,3.226703e-02,1.077710e-02,3.599521e-03,1.202230e-03,4.015414e-04,1.341137e-04,4.479359e-05
7,6.427876e-01,2.146893e-01,7.170561e-02,2.394947e-02,7.999056e-03,2.671662e-03,8.923276e-04,2.980349e-04,9.954282e-05,3.324702e-05
8,3.420201e-01,1.142338e-01,3.815376e-02,1.274325e-02,4.256209e-03,1.421562e-03,4.747976e-04,1.585811e-04,5.296563e-05,1.769037e-05
9,1.224647e-16,4.090286e-17,1.366144e-17,4.562882e-18,1.523990e-18,5.090083e-19,1.700074e-19,5.678198e-20,1.896502e-20,6.334263e-21
